In [1]:
# Import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import seaborn as sns
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error
# from statsmodels.tsa.holtwinters import SimpleExpSmoothing

In [2]:
# Load data
df = pd.read_excel('../data.xlsx', sheet_name='Rekapan')

In [ ]:
# Muncul data awal
df.head()

In [ ]:
# Muncul data akhir
df.tail()

In [ ]:
# Statistik deskriptif dasar
df.describe()

# Cek nilai yang hilang
df.isnull().sum()

# Visualisasi distribusi data
df.hist(bins=50, figsize=(20, 15))
plt.show()

In [ ]:
# Filter kolom terkait
data_filtered = df[['Tanggal', 'SO', 'TERKIRIM', 'Harga Komoditas Bijih Besi', 'Indeks Produksi Dalam Negeri', 'Data Inflasi', 'Kurs']]

# Ganti nama kolom
data_filtered.columns = ['Tanggal', 'SO', 'Terkirim', 'Harga Komoditas', 'Indeks Produksi', 'Data Inflasi', 'Kurs']

# Konversi tipe data
data_filtered['Tanggal'] = pd.to_datetime(data_filtered['Tanggal'])

# Ganti tipe data 'Indeks Produksi'
data_filtered['Indeks Produksi'] = pd.to_numeric(data_filtered['Indeks Produksi'], errors='coerce')

# Buat kolom 'Year' dan 'Month'
# data_filtered['Year'] = data_filtered['Tanggal'].dt.year
# data_filtered['Month'] = data_filtered['Tanggal'].dt.month

data_filtered['bulan_tahun'] = data_filtered['Tanggal'].dt.to_period('M')

In [ ]:
# Rekapan Bulanan
# monthly_summary = data_filtered.groupby(['Year', 'Month']).agg({
#     'SO': 'sum',
#     # 'Terkirim': 'sum',
#     # 'Harga Komoditas': 'mean',
#     # 'Indeks Produksi': 'mean',
#     # 'Data Inflasi': 'mean',
#     # 'Kurs': 'mean'
# }).reset_index()

monthly_summary = data_filtered.groupby('bulan_tahun').agg({
    'SO': 'sum',
    # Uncomment other columns as needed
    # 'Terkirim': 'sum',
    # 'Harga Komoditas': 'mean',
    # 'Indeks Produksi': 'mean',
    # 'Data Inflasi': 'mean',
    # 'Kurs': 'mean'
}).reset_index()

# Muncul Rekapan Bulanan
monthly_summary

In [ ]:
# Set alpha value for SES
alpha = 0.1

# Initialize columns for SES forecast and error metrics
monthly_summary['SES Forecast'] = np.nan
monthly_summary['Error'] = np.nan
monthly_summary['MAD'] = np.nan
monthly_summary['MSE'] = np.nan
monthly_summary['MAPE'] = np.nan

# Set initial forecast to the first SO value
monthly_summary.loc[0, 'SES Forecast'] = monthly_summary.loc[0, 'SO']

# Apply SES formula and calculate error metrics for each subsequent row
for i in range(1, len(monthly_summary)):
    # SES Forecast
    monthly_summary.loc[i, 'SES Forecast'] = (alpha * monthly_summary.loc[i-1, 'SO']) + ((1 - alpha) * monthly_summary.loc[i-1, 'SES Forecast'])
    
    # Error
    monthly_summary.loc[i, 'Error'] = monthly_summary.loc[i, 'SO'] - monthly_summary.loc[i, 'SES Forecast']
    
    # MAD (Mean Absolute Deviation)
    monthly_summary.loc[i, 'MAD'] = abs(monthly_summary.loc[i, 'Error'])
    
    # MSE (Mean Squared Error)
    monthly_summary.loc[i, 'MSE'] = monthly_summary.loc[i, 'Error'] ** 2
    
    # MAPE (Mean Absolute Percentage Error)
    if monthly_summary.loc[i, 'SO'] != 0:
        monthly_summary.loc[i, 'MAPE'] = (abs(monthly_summary.loc[i, 'Error']) / monthly_summary.loc[i, 'SO']) * 100
    else:
        monthly_summary.loc[i, 'MAPE'] = np.nan

# Show the result
print(monthly_summary)

In [ ]:
# Optional: Plot the actual vs forecasted values
plt.figure(figsize=(10, 6))
plt.plot(monthly_summary['bulan_tahun'].astype(str), monthly_summary['SO'], label='Actual SO', marker='o')
plt.plot(monthly_summary['bulan_tahun'].astype(str), monthly_summary['SES Forecast'], label='SES Forecast', marker='x')
plt.xlabel('Month-Year')
plt.ylabel('SO')
plt.xticks(rotation=45)
plt.title('Actual SO vs SES Forecast')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Function to generate future forecasts
def generate_future_forecasts(data, periods, alpha):
    last_actual = data['SO'].iloc[-1]
    last_forecast = data['SES Forecast'].iloc[-1]
    
    future_dates = pd.date_range(start=data['bulan_tahun'].iloc[-1].to_timestamp() + pd.DateOffset(months=1), periods=periods, freq='M')
    future_forecasts = []
    
    for _ in range(periods):
        next_forecast = alpha * last_actual + (1 - alpha) * last_forecast
        future_forecasts.append(next_forecast)
        last_actual = next_forecast  # Assume forecast becomes the actual for next period
        last_forecast = next_forecast
    
    future_df = pd.DataFrame({
        'bulan_tahun': future_dates.to_period('M'),
        'SO': np.nan,
        'SES Forecast': future_forecasts
    })
    
    return future_df

# Generate 12 months of future forecasts
future_periods = 12
future_forecasts = generate_future_forecasts(monthly_summary, future_periods, alpha)

# Combine historical data with future forecasts
combined_data = pd.concat([monthly_summary, future_forecasts]).reset_index(drop=True)

# Plot the results including future forecasts
plt.figure(figsize=(15, 8))
plt.plot(combined_data.index, combined_data['SO'], label='Actual SO', marker='o')
plt.plot(combined_data.index, combined_data['SES Forecast'], label='SES Forecast', marker='x')
plt.axvline(x=len(monthly_summary) - 1, color='r', linestyle='--', label='Forecast Start')
plt.xlabel('Month-Year')
plt.ylabel('SO')
plt.title('Actual SO vs SES Forecast (Including Future Predictions)')
plt.legend()
plt.xticks(range(0, len(combined_data), 2), combined_data['bulan_tahun'].astype(str)[::2], rotation=45)
plt.tight_layout()
plt.show()

# Display the future forecasts
print("\nFuture Forecasts:")
print(future_forecasts)

# Calculate and display overall error metrics
overall_mad = monthly_summary['MAD'].mean()
overall_mse = monthly_summary['MSE'].mean()
overall_mape = monthly_summary['MAPE'].mean()

print(f"\nOverall MAD: {overall_mad:.2f}")
print(f"Overall MSE: {overall_mse:.2f}")
print(f"Overall MAPE: {overall_mape:.2f}%")